In [1]:
!gdown --id 1lnhyUbxGjwSu8V-1KUFlVBKYJSrndTPe

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1lnhyUbxGjwSu8V-1KUFlVBKYJSrndTPe
To: /content/truyen_kieu_data.txt
100% 140k/140k [00:00<00:00, 70.1MB/s]


In [2]:
import string as pystring
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Lambda

In [3]:
f = open('truyen_kieu_data.txt', 'r')
lines = f.readlines()

In [4]:
PUNCT_TO_REMOVE = pystring.punctuation + pystring.digits + "\n"
def clean_text(text):
  text = text.lower()
  translator = str.maketrans(' ', ' ', PUNCT_TO_REMOVE)
  return text.translate(translator)
clean_text(lines[0])

'trăm năm trong cõi người ta'

In [5]:
def get_centers_and_contexts(corpus, max_window_size = 2):
  centers, contexts = [], []
  for line in corpus:
    line = clean_text(line)
    words = line.split()

    if len(words) <= 2*max_window_size:
      continue

    for i in range(max_window_size, len(words) - max_window_size):
      centers.append(words[i])
      idxs = list(range(i-max_window_size, i)) + list(range(i+1, i+1+max_window_size))
      contexts.append(" ".join([words[idx] for idx in idxs]))
  return centers, contexts


In [6]:
centers, contexts = get_centers_and_contexts(lines)
len(centers), len(contexts)

(9778, 9778)

In [7]:
max_lenghth = 4
embedding_size = 200

In [8]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(lines)

In [9]:
vocab_size = len(tokenizer.word_index) + 1

In [10]:
train_sequences = tokenizer.texts_to_sequences(contexts)
train_sequences = pad_sequences(train_sequences, maxlen=max_lenghth, truncating='post', padding='post')

In [11]:
train_labels = [to_categorical(tokenizer.word_index[label], len(tokenizer.word_index) + 1) for label in centers]
train_labels = np.array(train_labels)

In [12]:
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_lenghth))
cbow.add(Lambda(lambda x: tf.reduce_mean(x, axis=1), output_shape=(embedding_size,)))
cbow.add(Dense(units=vocab_size, activation='softmax'))
cbow.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 200)            627200    
                                                                 
 lambda (Lambda)             (None, 200)               0         
                                                                 
 dense (Dense)               (None, 3136)              630336    
                                                                 
Total params: 1257536 (4.80 MB)
Trainable params: 1257536 (4.80 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
cbow.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cbow.fit(train_sequences, train_labels, epochs=50, verbose=1)

Epoch 1/50
306/306 [==============================] - 7s 20ms/step - loss: 7.5908 - accuracy: 0.0193
Epoch 2/50
306/306 [==============================] - 7s 23ms/step - loss: 6.7520 - accuracy: 0.0204
Epoch 3/50
306/306 [==============================] - 6s 18ms/step - loss: 6.6073 - accuracy: 0.0230
Epoch 4/50
306/306 [==============================] - 7s 22ms/step - loss: 6.4636 - accuracy: 0.0267
Epoch 5/50
306/306 [==============================] - 6s 20ms/step - loss: 6.2605 - accuracy: 0.0415
Epoch 6/50
306/306 [==============================] - 6s 20ms/step - loss: 5.9787 - accuracy: 0.0620
Epoch 7/50
306/306 [==============================] - 7s 22ms/step - loss: 5.6150 - accuracy: 0.0869
Epoch 8/50
306/306 [==============================] - 6s 19ms/step - loss: 5.1829 - accuracy: 0.1264
Epoch 9/50
306/306 [==============================] - 7s 23ms/step - loss: 4.7027 - accuracy: 0.1798
Epoch 10/50
306/306 [==============================] - 6s 18ms/step - loss: 4.2083 - accura

In [14]:
sample_text = "ghen thua liễu hờn"
sample_seq = tokenizer.texts_to_sequences([sample_text])
sample_seq = pad_sequences(sample_seq, maxlen=max_lenghth, truncating='post', padding='post')
cbow.predict(sample_seq)

1/1 [==============================] - 0s 93ms/step


array([[2.9561933e-09, 3.1412744e-09, 3.2273795e-10, ..., 3.0202139e-09,
        3.3870304e-09, 2.6247944e-08]], dtype=float32)

In [15]:
tokenizer.index_word[np.argmax(cbow.predict(sample_seq))]

1/1 [==============================] - 0s 20ms/step


'thắm'